# Generador de personas aleatorias

In [138]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [193]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		clave_estado_nac = clave_estados[estado_nac]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'Mexico'):
	# Hacer peticion a un api para obtener coordenadas del centro y extremas
	# querystring = urllib.parse.urlencode({'city': ciudad, 'state': estado, 'country': pais})
	querystring = urllib.parse.urlencode({'q': f'{ciudad} {estado} {pais}'})
	try:
		df = pd.read_json(f'https://nominatim.openstreetmap.org/search?{querystring}&polygon_geojson=1&format=json')
		# En caso de que el dataframe este vacio
		if df.empty: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		# Obtener coordenadas del centro
		centro = (df['lat'][0], df['lon'][0])
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in df['boundingbox'][0]]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return None

	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(centro, extremo_so, extremo_ne, n = 1, reducir = True):
	# Obtener coordenadas del centro
	cenx, ceny = centro
	# Obtener extremos originales
	minx, miny = extremo_so
	maxx, maxy = extremo_ne
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random):
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		direccion = geolocator.reverse(punto_random, language = 'es_MX')
		# print(direccion.address)
		direccion_raw = direccion.raw
		direccion = direccion[0].split(', ')
		print(direccion)
		print(direccion_raw['address'])
		count = len(direccion)
		if count <= 2: continue
		if count == 10:
			print('\033[34mHay de 10\033[0m')
			print(list(direccion))
		elif count == 9:
			print('\033[34mHay de 9\033[0m')
			print(list(direccion))
			# [
			#   'Jardin De Niños Gabino Barrera',
			#   'Avenida Azarco',
			#   'Villa Insurgentes (El Calabazal)',
			#   'Barrio El Cerrito',
			#   'villa insurgentes  (El Calabazal)',
			#   'Sombrerete',
			#   'Zacatecas',
			#   '99109',
			#   'México'
			# ]
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 8:
			dirr['local'] = direccion[0]
			if direccion[1].isnumeric(): dirr['numero'] = direccion[1]
			else: dirr['numero'] = rm.randint(1, 999)
			dirr['calle'] = direccion[2]
			dirr['colonia'] = direccion[3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 7:
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[2]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 6:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 5:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			dirr['ciudad'] = direccion[-3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['pais'] = direccion[-1]
			listo = True
		else:
			continue
		if listo:
			dirr['arr_original'] = direccion
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			dirrr = dirr
			break
	if not listo:
		print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion')
	return dirrr

# geolocator = Nominatim(user_agent = 'geoapiExercises')
# location = geolocator.geocode('')
# punto_random = ()
# direccion = geolocator.reverse(punto_random, language = 'es_MX')

In [194]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		print('Idx', _)
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].to_dict(orient = 'records')[
			0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		tel = f'({ct_ciudad["lada"]})' + ' ' + '{:03d}'.format(rm.randint(0, 999)) + ' ' + '{:04d}'.format(rm.randint(0, 9999))

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = suroeste = noreste = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			suroeste = eval(ct_ciudad['suroeste'])
			noreste = eval(ct_ciudad['noreste'])
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					suroeste = coordenadas['suroeste']
					noreste = coordenadas['noreste']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(suroeste)
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(noreste)
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {_}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and suroeste and noreste:
			puntos_random = generar_puntos_random(centro, suroeste, noreste, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				if direccion and all(k in direccion for k in ('calle', 'numero', 'colonia', 'cp')):
					calle = direccion['calle']
					numero = direccion['numero']
					colonia = direccion['colonia']
					cp = direccion['cp']
					direcc = f'{calle} #{numero}, {colonia}'
			except BaseException as be:
				print(f'\033[31mAddressException: idx {_}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx 0
obtener_coordenadas(): Coordenadas encontradas en "Venustiano Carranza", "Ciudad de Mexico", "Mexico"
['Calle Oriente 166', 'Colonia Moctezuma 2a. Sección', 'Venustiano Carranza', 'Ciudad de México', '15530', 'México']
{'road': 'Calle Oriente 166', 'neighbourhood': 'Colonia Moctezuma 2a. Sección', 'state': 'Ciudad de México', 'postcode': '15530', 'country': 'México', 'country_code': 'mx'}
buscar_direccion(): Direccion encontrada en (19.427432490247572, -99.09854583557947)
Idx 1
obtener_coordenadas(): Coordenadas encontradas en "Berriozabal", "Chiapas", "Mexico"
['Berriozábal', 'Chiapas', 'México']
{'county': 'Berriozábal', 'state': 'Chiapas', 'country': 'México', 'country_code': 'mx'}
['Berriozábal', 'Chiapas', 'México']
{'county': 'Berriozábal', 'state': 'Chiapas', 'country': 'México', 'country_code': 'mx'}
['Berriozábal', 'Chiapas', 'México']
{'county': 'Berriozábal', 'state': 'Chiapas', 'country': 'México', 'country_code': 'mx'}
['Berriozábal', 'Chiapas', 'México']
{'county': 

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Wendy,Casas,Godoy,2006-04-24,M,151.79,48.22,O+,CAGW060424MDFAOES9,...,None,None,"Calle Oriente 166 #661, Colonia Moctezuma 2a. ...",15530,Venustiano Carranza,Ciudad de Mexico,19.419232,-99.113395,America/Mexico_City,-0600
1,Ing.,Mónica,Mojica,Casanova,1994-02-15,M,158.71,85.44,O-,MOCM940215MCSOAO22,...,Oficial de educación comunitaria,64 63 94 9429 1,None,None,Berriozabal,Chiapas,16.871689,-93.337076,America/Mexico_City,-0600
2,None,Gabriel,Zapata,Posada,2007-02-09,H,160.41,69.97,A+,ZAPG070209HGRAOAW6,...,None,None,"Calle General Hermenegildo Galeana #657, Petac...",91316,Petacalco,Guerrero,17.980556,-102.108056,America/Mexico_City,-0600
3,None,Amalia,Zaragoza,Muro,1934-09-03,M,164.57,48.92,A+,ZAMA340903MCLAUM96,...,Gerente de TI,69 86 34 3826 3,"Avenida Cuauhtémoc #151, Las Esperanzas",97000,Las Esperanzas,Coahuila,27.741752,-101.352427,America/Monterrey,-0600
4,Mtro.,Víctor,Quesada,Robledo,1956-05-24,H,186.41,86.80,A+,QURV560524HCCUOI92,...,Administrador de deportes,48 49 56 2619 6,"Calle 31 #581, Bécal",80837,Becal,Campeche,20.441473,-90.028359,America/Merida,-0600
5,Mtra.,Jaqueline,Báez,Jáquez,1916-03-06,M,162.93,47.05,O+,BAJJ160306MJCAAA82,...,Desarrollador de juegos de computadora,10 16 16 7114 2,"Mechoacanejo #854, Teocaltiche",47215,Mechoacanejo,Jalisco,21.541154,-102.598126,America/Mexico_City,-0600
6,Dr.,Fernando,Arteaga,Rodríquez,1933-09-13,H,168.75,64.82,A+,ARRF330913HGRROE40,...,Director de arte publicitario,88 49 33 8709 0,None,None,Acuchillán del Progreso,Guerrero,NaN,NaN,None,None
7,Sr.,Rogerio,Colón,Espinal,1916-03-10,H,182.89,83.22,O+,COER160310HQROSO69,...,Gerente de TI,11 92 16 4902 2,"San Miguel de Cozumel #374, Cozumel",77645,Cozumel,Quintana Roo,20.431759,-86.920275,America/Cancun,-0500
8,None,Nicolás,Chapa,León,2017-03-02,H,107.76,22.80,O+,CHLN170302HQRHEIN0,...,None,None,None,None,José María Morelos,Quintana Roo,19.707179,-88.778134,America/Cancun,-0500
9,None,Espartaco,Portillo,Calderón,1916-10-23,H,186.96,77.90,A+,POCE161023HTSOAS74,...,Locutor de la transmisión,46 90 16 1440 2,"Calle Cuarta #336, Ignacio Manuel Altamirno",84217,Villa Manuel,Tamaulipas,25.892865,-97.818327,America/Matamoros,-0500
